# Deploy FIWARE and Smart Village microservices

Have you heard the joke, how microservices were invented? A software engineer had a task to take an existing monolithic software application and split it into microservices. The engineer connected 5 cell phones to the existing application, glued each phone to a picture frame, and showed the application running on all 5 phones. Real microservices run in modern cloud environments like Red Hat OpenShift. 

A working cloud project that is capable of receiving data from IoT smart devices and processing edge device data is composed of several important microservices. We will deploy each of the required microservices below to begin sending, receiving, and analyzing IoT edge device data. Then in later notebooks, we will review each microservice and how it works inside of the Smart Village Platform. 

## Deploy the MongoDB NOSQL Database in NERC OpenShift

A MongoDB No-SQL Database is required for FIWARE IoT Agents to store smart device registration data about each smart device in the project. 

Run the Ansible Playbook below to deploy MongoDB to the cloud. You can review the [03-about-mongodb.ipynb](03-about-mongodb.ipynb) notebook later in the course to troubleshoot, or find more details about the deployment of MongoDB. 

In [ ]:
%%bash
ansible-playbook ~/smartvillage-operator/apply-edgemongodb.yaml \
  -e ansible_operator_meta_namespace=smart-village-faeeb6c \
  -e crd_path=~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/edgemongodbs/default/edgemongodb.yaml
echo DONE

### View MongoDB pod details
After running the Ansible Playbook, it will take a minute before the MongoDB pod is up and running. Run the command below until the MongoDB pod health checks are `READY 2/2` and `STATUS Running`. 

In [ ]:
%%bash
oc -n smart-village-faeeb6c get pod -l app.kubernetes.io/instance=mongodb
oc -n smart-village-faeeb6c wait pod -l app.kubernetes.io/instance=mongodb --for=condition=Ready --timeout=2m
oc -n smart-village-faeeb6c get pod -l app.kubernetes.io/instance=mongodb
echo DONE

## Deploy the RabbitMQ in the NERC OpenShift
A message broker like RabbitMQ can receive messages from smart devices and forward them on to an IoT Agent to process the message. 

Run the Ansible Playbook below to deploy RabbitMQ to the cloud. You can review the [04-about-rabbitmq.ipynb](04-about-rabbitmq.ipynb) notebook later in the course to troubleshoot, or find more details about the deployment of RabbitMQ. 

In [ ]:
%%bash
ansible-playbook ~/smartvillage-operator/apply-edgerabbitmq.yaml \
  -e ansible_operator_meta_namespace=smart-village-faeeb6c \
  -e crd_path=~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/edgerabbitmqs/rabbitmq/edgerabbitmq.yaml
echo DONE

### View RabbitMQ pod details
After running the Ansible Playbook, it will take a minute before the RabbitMQ pod is up and running. Run the command below until the RabbitMQ pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
%%bash
oc -n smart-village-faeeb6c get pod -l app.kubernetes.io/name=rabbitmq
oc -n smart-village-faeeb6c wait pod -l app.kubernetes.io/name=rabbitmq --for=condition=Ready --timeout=2m
oc -n smart-village-faeeb6c get pod -l app.kubernetes.io/name=rabbitmq
echo DONE

## Deploy PostgreSQL in NERC OpenShift
A relational SQL database like PostgreSQL is required by the ScorpioBroker to store smart device context data for each smart device. We also store configurable user data like traffic simulations, simulation reports, and more in the Smart Village application with PostgreSQL. 

Run the command below to deploy the database create SQL scripts as a Kubernetes ConfigMap to the cloud. These scripts will be run later, when the PostgreSQL pod is running to initialize the Smart Village database. 

In [ ]:
%%bash
git clone https://github.com/smartabyar-smartvillage/smartabyar-smartvillage.git ~/smartabyar-smartvillage/
oc -n smart-village-faeeb6c create configmap smartvillage-db-create --from-file ~/smartabyar-smartvillage/src/main/resources/sql/db-create.sql
echo DONE

Run the Ansible Playbook below to deploy PostgreSQL to the cloud. You can review the [05-about-postgres.ipynb](05-about-postgres.ipynb) notebook later in the course to troubleshoot, or find more details about the deployment of PostgreSQL. 

In [ ]:
%%bash
ansible-playbook ~/smartvillage-operator/apply-edgepostgres.yaml \
  -e ansible_operator_meta_namespace=smart-village-faeeb6c \
  -e crd_path=~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/edgepostgress/postgres/edgepostgres.yaml
echo DONE

You should see a play recap that has failed. 
This is expected because the postgres pod is just now getting created. 
The final tasks in the playbook expect the database create SQL scripts to be run for the smartvillage application in postgres. 

Retry the playbook once the postgres pod is running. 


### View PostgreSQL pod details
After running the Ansible Playbook, it will take a minute before the PostgreSQL pod is up and running. Run the command below until the PostgreSQL pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
%%bash
oc -n smart-village-faeeb6c get pod -l postgres-operator.crunchydata.com/instance-set=postgres
oc -n smart-village-faeeb6c wait pod -l postgres-operator.crunchydata.com/instance-set=postgres --for=condition=Ready --timeout=2m
oc -n smart-village-faeeb6c get pod -l postgres-operator.crunchydata.com/instance-set=postgres
echo DONE

Now re-run the ansible Playbook to complete the last tasks for PostgreSQL. 

In [ ]:
%%bash
ansible-playbook ~/smartvillage-operator/apply-edgepostgres.yaml \
  -e ansible_operator_meta_namespace=smart-village-faeeb6c \
  -e crd_path=~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/edgepostgress/postgres/edgepostgres.yaml
echo DONE

### Review the Ansible Playbook for PostgreSQL
After running the Ansible Playbook, you should see a PLAY RECAP where no tasks have failed `failed=0`. 

## Deploy the AMQ Streams in the NERC OpenShift
A message broker like AMQ Streams can send and receive messages in the ScorpioBroker and Smart Village Platform. 

Run the Ansible Playbook below to deploy AMQ Streams to the cloud. 

In [ ]:
%%bash
ansible-playbook ~/smartvillage-operator/apply-edgekafka.yaml \
  -e ansible_operator_meta_namespace=smart-village-faeeb6c \
  -e crd_path=~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/edgekafkas/default/edgekafka.yaml
echo DONE

### View AMQ Streams pod details
After running the Ansible Playbook, it will take a minute before kafka pods are up and running. Run the command below until the kafka pods health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
%%bash
oc -n smart-village-faeeb6c get pod -l app.kubernetes.io/name=kafka
oc -n smart-village-faeeb6c wait pod -l app.kubernetes.io/name=kafka --for=condition=Ready --timeout=2m
oc -n smart-village-faeeb6c get pod -l app.kubernetes.io/name=kafka
echo DONE

## Deploy the ScorpioBroker Context Broker in NERC OpenShift
A FIWARE Context Broker like ScorpioBroker is required for storing smart device entity data of vehicle traffic cameras and pedestrian traffic cameras in the project. 

Run the Ansible Playbook below to deploy ScorpioBroker to the cloud. You can review the [06-about-scorpiobroker.ipynb](06-about-scorpiobroker.ipynb) notebook later in the course to troubleshoot, or find more details about the deployment of ScorpioBroker. 

In [ ]:
%%bash
ansible-playbook ~/smartvillage-operator/apply-scorpiobroker.yaml \
  -e ansible_operator_meta_namespace=smart-village-faeeb6c \
  -e crd_path=~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/scorpiobrokers/scorpiobroker/scorpiobroker.yaml
echo DONE

### View ScorpioBroker pod details
After running the Ansible Playbook, it will take a minute before the ScorpioBroker pod is up and running. Run the command below until the ScorpioBroker pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
%%bash
oc -n smart-village-faeeb6c get pod -l app.kubernetes.io/instance=scorpiobroker
oc -n smart-village-faeeb6c wait pod -l app.kubernetes.io/instance=scorpiobroker --for=condition=Ready --timeout=2m
oc -n smart-village-faeeb6c get pod -l app.kubernetes.io/instance=scorpiobroker
echo DONE

## Deploy the IoT Agent JSON in NERC OpenShift
A FIWARE IoT Agent like IoT Agent JSON is required for smart device registration of vehicle traffic cameras and pedestrian traffic cameras in the project. 

Run the Ansible Playbook below to deploy IoT Agent to the cloud. You can review the [07-about-iot-agent.ipynb](07-about-iot-agent.ipynb) notebook later in the course to troubleshoot, or find more details about the deployment of IoT Agent. 

In [ ]:
%%bash
ansible-playbook ~/smartvillage-operator/apply-iotagentjson.yaml \
  -e ansible_operator_meta_namespace=smart-village-faeeb6c \
  -e crd_path=~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/iotagentjsons/iotagent-json/iotagentjson.yaml
echo DONE

### View IoT Agent pod details
After running the Ansible Playbook, it will take a minute before the IoT Agent pod is up and running. Run the command below until the IoT Agent pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
%%bash
oc -n smart-village-faeeb6c get pod -l app.kubernetes.io/instance=iotagent-json
oc -n smart-village-faeeb6c wait pod -l app.kubernetes.io/instance=iotagent-json --for=condition=Ready --timeout=2m
oc -n smart-village-faeeb6c get pod -l app.kubernetes.io/instance=iotagent-json
echo DONE

## Deploy Zookeeper in NERC OpenShift
A cluster manager like Apache Zookeeper is required for distributing messages and workloads to multiple pods of reactive/asynchronous microservices like Apache Solr and the Smart Village Platform. 

Run the Ansible Playbook below to deploy Zookeeper to the cloud. You can review the [08-about-zookeeper.ipynb](08-about-zookeeper.ipynb) notebook later in the course to troubleshoot, or find more details about the deployment of Zookeeper. 

In [ ]:
%%bash
ansible-playbook ~/smartvillage-operator/apply-edgezookeeper.yaml \
  -e ansible_operator_meta_namespace=smart-village-faeeb6c \
  -e crd_path=~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/edgezookeepers/default/edgezookeeper.yaml
echo DONE

### View Zookeeper pod details
After running the Ansible Playbook, it will take a minute before the Zookeeper pod is up and running. Run the command below until the Zookeeper pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
%%bash
oc -n smart-village-faeeb6c get pod -l app=zookeeper
oc -n smart-village-faeeb6c wait pod -l app=zookeeper --for=condition=Ready --timeout=2m
oc -n smart-village-faeeb6c get pod -l app=zookeeper
echo DONE

## Deploy Solr in NERC OpenShift
An open source search engine like Apache Solr is required by the Smart Village application API to serve up API stored objects as quickly as possible. APIs that are backed by a search engine have numerous additional benefits compared to an API backed by a traditional relational database. A search engine is always indexed for the fastest data retrieval possible. The advanced data and query parsing of a search engine allows for extremely fast full text search, filtering, and sorting of the data. Search engine data can also be grouped, faceted, and pivoted on, for an advanced set of analytics and statistics on specific data in your query. 

Run the command below to deploy the default computate Solr ConfigSet as a Kubernetes ConfigMap to the cloud. These configsets will be loaded later, when the Solr pod is running to initialize the Solr ConfigSet which is the schema used for Solr Collections in the Smart Village Platform. 

In [ ]:
%%bash
oc -n smart-village-faeeb6c apply -f ~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/edgesolrs/default/configmaps/computate-configset/configmap.yaml
echo DONE

Run the Ansible Playbook below to deploy Solr to the cloud. You can review the [09-about-solr.ipynb](09-about-solr.ipynb) notebook later in the course to troubleshoot, or find more details about the deployment of Solr. 

In [ ]:
%%bash
ansible-playbook ~/smartvillage-operator/apply-edgesolr.yaml \
  -e ansible_operator_meta_namespace=smart-village-faeeb6c \
  -e crd_path=~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/edgesolrs/default/edgesolr.yaml
echo DONE

You should see a play recap that has failed. 
This is expected because the Solr pod is just now getting created. 
The final tasks in the playbook expect the solr pod to be running to connect to the running pod and create the Solr ConfigSets and Solr Collections used by the Smart Village application. 

Retry the playbook once the Solr pod is running. 


### View Solr pod details
After running the Ansible Playbook, it will take a minute before the Solr pod is up and running. Run the command below until the Solr pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
%%bash
oc -n smart-village-faeeb6c get pod -l app=solr
oc -n smart-village-faeeb6c wait pod -l app=solr --for=condition=Ready --timeout=2m
oc -n smart-village-faeeb6c get pod -l app=solr
echo DONE

When the Solr pod is running, run the Ansible Playbook again. 

In [ ]:
%%bash
ansible-playbook ~/smartvillage-operator/apply-edgesolr.yaml \
  -e ansible_operator_meta_namespace=smart-village-faeeb6c \
  -e crd_path=~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/edgesolrs/default/edgesolr.yaml
echo DONE

## Deploy the SmartaByarSmartVillage in NERC OpenShift
The Smart Village Platform is used by researchers to configure smart devices for improving traffic light configuration, and running on reports on traffic simulations configured at intersections in the world. 

Run the Ansible Playbook below to deploy the Smart Village Platform to the cloud. You can review the [10-about-smartvillage.ipynb](10-about-smartvillage.ipynb) notebook later in the course to troubleshoot, or find more details about the deployment of the Smart Village Platform. 


In [ ]:
%%bash
ansible-playbook ~/smartvillage-operator/apply-smartabyarsmartvillage.yaml \
  -e ansible_operator_meta_namespace=smart-village-faeeb6c \
  -e crd_path=~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/smartabyarsmartvillages/smartvillage/smartabyarsmartvillage.yaml
echo DONE

### View Smart Village pod details
After running the Ansible Playbook, it will take a minute before the Smart Village pod is up and running. Run the command below until the Smart Village pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
%%bash
oc -n smart-village-faeeb6c get pod -l app=smartabyar-smartvillage-web
oc -n smart-village-faeeb6c wait pod -l app=smartabyar-smartvillage-web --for=condition=Ready --timeout=5m
oc -n smart-village-faeeb6c get pod -l app=smartabyar-smartvillage-web
echo DONE

## Next...
If you have successfully ran all of the commands above, congratulations, you are ready to move on to the next notebook in the course. 
- If you have additional questions or issues, please [create an issue for the course here](https://github.com/smartabyar-smartvillage/smartabyar-smartvillage-nerc-course/issues). 
- Otherwise, please continue to the next notebook [03-about-mongodb.ipynb](03-about-mongodb.ipynb). 
